# Exercise 3 - Huffman codes

| Lorenzo | Calandra Buonaura | 2107761 |
|---------|-------------------|---------|

In [23]:
import numpy as np
import pandas as pd

In [24]:
with open("text.txt", "r") as file:
  all_characters = file.read()
  
unique_characters = sorted(list(set(all_characters)))
n = len(all_characters)

character_probabilities = {char: all_characters.count(char)/n for char in unique_characters}

In [25]:
probability_df = pd.DataFrame(list(character_probabilities.items()), columns=["Character", "Probability"])
probability_df['Error'] = np.sqrt((probability_df['Probability'] * (n - probability_df['Probability'])) / n**2)
print(probability_df.head())

  Character  Probability     Error
0        \n     0.041855  0.000628
1               0.224531  0.001455
2         !     0.002280  0.000147
3         &     0.000028  0.000016
4         '     0.005550  0.000229


In [26]:
def shannon_entropy(df):
  entropy = -sum(df['Probability'] * np.log2(df['Probability']))
  error = sum(np.abs(np.log2(df['Probability']) + 1/np.log(2))*df['Error'])
  return entropy, error

In [27]:
entropy, error_entropy = shannon_entropy(probability_df)
print(f"Shannon entropy of the file \'text.txt\': {round(entropy, 4)} +/- {round(error_entropy, 4)}")

Shannon entropy of the file 'text.txt': 4.5762 +/- 0.0815


In [96]:
def huffman_code(df):
  df['Code'] = df['Character'].apply(lambda _: [])
  new_df = df.copy()
  p = new_df['Probability'].to_numpy()
  
  while True:
    min_value1 = np.min(p)
    min_indices1 = np.where(p == min_value1)[0]
    for index in min_indices1:
      df['Code'][index].append(0)
    p[min_indices1] = np.inf

    min_value2 = np.min(p)
    min_indices2 = np.where(p == min_value2)[0]
    for index in min_indices2:
      df['Code'][index].append(1)
    p[min_indices2] = np.inf
    
    p[min_indices1] = min_value1 + min_value2
    p[min_indices2] = min_value2 + min_value2
    
    if abs(min_value1 + min_value2 - 1) <= 0.01:
      break
    
    df['Code'] = df['Code'].apply(lambda code: code[::-1])
    
  return df

def build_code_dict(df):
  code_dict = {}
  for _, row in df.iterrows():
    code_tuple = tuple(row['Code'])
    code_dict[code_tuple] = row['Character']
  return code_dict

def huffman_encode(input_string, code_dict):
  encoded_output = []
  for char in input_string:
    for code_tuple, character in code_dict.items():
      if character == char:
        encoded_output.extend(code_tuple)
        break
  return encoded_output
  
def huffman_decode(encoded_sequence, code_dict):
  decoded_output = []
  current_code = []
  
  for bit in encoded_sequence:
    current_code.append(bit)
    code_tuple = tuple(current_code)
    
    if code_tuple in code_dict:
      decoded_output.append(code_dict[code_tuple])
      print(code_tuple)
      current_code = []
  return decoded_output

In [97]:
new_df = huffman_code(probability_df)
code_dict = build_code_dict(new_df)

In [98]:
encoded_sequence = huffman_encode(all_characters[:90], code_dict)

In [99]:
print(all_characters[:90])

ACT I
SCENE I. On a ship at sea: a tempestuous noise

    of thunder and lightning heard.



In [100]:
decoded_output = huffman_decode(encoded_sequence, code_dict)

(0, 0, 1, 1, 1, 1, 0, 1)
(0, 1, 1, 1)


In [95]:
decoded_output

[]